In [ ]:
"""
Created on Sat Nov 13, 14:30:00 2021
@authors: ndiop, odiop

Aim: extraction of all links and affect them to charging stations. 
   Inputs: - a network from in which the zone is to be extracted
           - a csv file for charging stations where X and Y cordonates are given (obtained with QGIS) 
   Output: a csv file containing all charging stations and links affected to them
"""

In [1]:
import math
import numpy as np
import pandas as pd
import matsim

# Input data

In [3]:
net = matsim.read_network("multimodalnetwork_truck.xml") # network 
B=pd.read_csv("bornesXY.csv", ",",encoding='utf8', dtype=str) # charging station with XY coordinates from QGIS

/home/ndiop/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
tabnode=net.nodes
tablink=net.links

In [9]:
tablink["modes"].value_counts()

car                                   294402
car,truck                             135193
bus,pt,car,truck                       24609
bus,car,truck                           9350
bus,pt,car                              6629
pt,rail,train                           6241
bus,car                                 6196
rail                                     931
bus,pt                                   586
bus,artificial,rail,light_rail           237
artificial,rail,light_rail,tram          174
stopFacilityLink,artificial,tram         171
stopFacilityLink,artificial,subway       124
artificial,subway,rail,light_rail        120
stopFacilityLink,bus,artificial           85
bus,pt,car,tram,truck                     17
pt                                         3
subway,pt                                  3
artificial,rail,light_rail                 1
Name: modes, dtype: int64

In [10]:
tabnode

,x,y,node_id
0,710081.402314,7.061403e+06,1000016733
1,709996.648774,7.061533e+06,1000017859
2,708894.203801,7.056854e+06,1000041036
3,709735.516999,7.035480e+06,1000082103
4,716530.345273,7.041756e+06,1000791305
...,...,...,...
218433,740906.111389,7.040142e+06,pt_transvilles_8911
218434,725453.672058,7.043689e+06,pt_transvilles_927
218435,732215.114114,7.044256e+06,pt_transvilles_9756
218436,720298.396974,7.027965e+06,pt_transvilles_A058


# Link preprocessing 

In [11]:
# Filter links with car occurency in modes
carLink = tablink[(tablink["modes"]=="car") | (tablink["modes"]=="car,truck") | 
                  (tablink["modes"]=="bus,pt,car,truck") | (tablink["modes"]=="bus,car,truck") | 
                  (tablink["modes"]=="bus,pt,car") | (tablink["modes"]=="bus,car") | 
                  (tablink["modes"]=="bus,pt,car,tram,truck")] 

                  # to check car link : tablink.modes.value_counts()
    
# Mapping link to nodes with there coordinates
def mapping_codeINSEE_xy(data, data_xy):
    # select columns
    data_xy.columns =['x', 'y', 'node_id']
    # create key mapping for x and y coordinates
    map_x = dict(zip(data_xy.node_id, data_xy.x))
    map_y = dict(zip(data_xy.node_id, data_xy.y))
    # create columns for data input to receive coordinates for node from and node to
    data["Xfrom"] = data["from_node"]
    data["Yfrom"] = data["from_node"]
    data["Xto"] = data["to_node"]
    data["Yto"] = data["to_node"]    
    # replace node_id by coordinates mapped
    data[["Xfrom","Xto"]] = data[["Xfrom","Xto"]].applymap(map_x.get)
    data[["Yfrom","Yto"]] = data[["Yfrom","Yto"]].applymap(map_y.get)
    # drop data with x, y not mapped
    data = data.dropna(subset=["Xfrom","Yfrom", "Xto","Yto"])
    # return data with new columns mapped
    return data
carLinkXY = mapping_codeINSEE_xy(carLink.copy(), tabnode.copy())
carLinkXY.reset_index(drop=True, inplace=True)

In [18]:
carLinkXY

,length,freespeed,capacity,permlanes,oneway,modes,link_id,from_node,to_node,Xfrom,Yfrom,Xto,Yto
0,22.317224,4.166667,600.0,1.0,1,car,1,1052159468,1052159497,603332.480516,7.063362e+06,603319.248102,7.063344e+06
1,412.971951,6.944444,600.0,1.0,1,car,10,563664699,1052159477,603213.879184,7.062753e+06,602855.279227,7.062581e+06
2,29.825621,4.166667,600.0,1.0,1,car,100,3933297330,365302866,680077.173716,7.060710e+06,680084.415466,7.060736e+06
3,121.791936,13.888889,1000.0,1.0,1,"car,truck",1000,940143173,3949596855,671567.356863,7.074174e+06,671542.794767,7.074063e+06
4,67.593558,4.166667,600.0,1.0,1,car,10000,972739550,1190271088,707304.783790,7.029529e+06,707243.116729,7.029557e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
476391,128.697633,13.888889,1500.0,1.0,1,"car,truck",99995,4935862050,5691780422,656701.772727,7.060218e+06,656696.027119,7.060344e+06
476392,20.249542,13.888889,1500.0,1.0,1,"car,truck",99996,5691780422,5691780423,656696.027119,7.060344e+06,656697.345566,7.060364e+06
476393,176.813521,6.944444,600.0,1.0,1,"car,truck",99997,766707133,766706951,667661.948290,7.042812e+06,667822.658035,7.042864e+06
476394,176.813521,6.944444,600.0,1.0,1,"car,truck",99998,766706951,766707133,667822.658035,7.042864e+06,667661.948290,7.042812e+06


# Reduced area

In [221]:
# Test function in Dunkerque
#https://data.opendatasoft.com/explore/dataset/base-officielle-des-codes-postaux%40cudunkerque/table/?disjunctive.code_commune_insee&disjunctive.nom_de_la_commune&disjunctive.code_postal&disjunctive.ligne_5
'''
codeCommDK = pd.read_csv("base-officielle-des-codes-postaux.csv", ";",encoding='utf8', dtype=str)
lDK = list(codeCommDK["Code_commune_INSEE"].unique())
s=list(B["code_insee"])
u=set(lDK).intersection(set(s))
u=list(u)
isDK = (B["code_insee"]==u[0]) | (B["code_insee"]==u[1])  # 2 communes (len(u))
Bdk = B[isDK]
Bdk.reset_index(drop=True, inplace=True)
Bdk.to_csv("bornesDK.csv", sep=";", encoding="utf8")
'''

'\ncodeCommDK = pd.read_csv("base-officielle-des-codes-postaux.csv", ";",encoding=\'utf8\', dtype=str)\nlDK = list(codeCommDK["Code_commune_INSEE"].unique())\ns=list(B["code_insee"])\nu=set(lDK).intersection(set(s))\nu=list(u)\nisDK = (B["code_insee"]==u[0]) | (B["code_insee"]==u[1])  # 2 communes (len(u))\nBdk = B[isDK]\nBdk.reset_index(drop=True, inplace=True)\nBdk.to_csv("bornesDK.csv", sep=";", encoding="utf8")\n'

In [19]:
# Test function in NPDC
isNPDC = (B.code_insee.str.startswith('59')) | (B.code_insee.str.startswith('62'))
Bnpdc = B[isNPDC]
Bnpdc.reset_index(drop=True, inplace=True)
Bnpdc.to_csv("bornesNPDC.csv", sep=";", encoding="utf8")

# Intermediate functions

In [6]:
# 1/ Pick coordinates of 2 nodes of each link from carLinkXY file
def GetCoordonnees(df,k):
    x1 = df.loc[k, "Xfrom"]
    y1 = df.loc[k, "Yfrom"]
    x2 = df.loc[k, "Xto"]
    y2 = df.loc[k, "Yto"]
    return x1,y1,x2,y2

# 2/ Calculate the coefficients of the line of the link
def Equationdroite(x1,x2,y1,y2): # equation droite lien
    cd = (y2-y1)/(x2-x1)
    oo = y1 - cd*x1
    return cd,oo

# 3/ Test if line of the link and circle C(xb,yb,rayon) have intersection to keep potential links
def isIntersection(x0, y0, r, a, b): # equation have at least one solution for it
    a1 = 1+ a*a
    b1 = 2*(-x0 + a*b - a*y0)
    c1 = x0*x0 - r*r + (b - y0)*(b - y0)
    delta = b1*b1 - 4*a1*c1
    return (delta>=0)

# 4/ Calculate corner between charging point and nodes of links for Test if the angle at the terminal with the vertices of the potential link is obtuse (>90)
def testCos(xb,yb,x1l,y1l,x2l,y2l):
    
    l=math.sqrt((x1l-x2l)*(x1l-x2l)+(y1l-y2l)*(y1l-y2l)) # length of link
    
    l1=math.sqrt((x1l-xb)*(x1l-xb)+(y1l-yb)*(y1l-yb)) # length between node 1 of link and charging point
    
    l2=math.sqrt((xb-x2l)*(xb-x2l)+(yb-y2l)*(yb-y2l)) # length between node 2 of link and charging point
    
    alpha = (-l*l + l1*l1 + l2*l2)/(2*l1*l2)
    
    return (np.arccos(alpha))

# 5/ Check the link more near to the charging station
def IntersectionDroite(a1,b1,a2,b2):
    x = (b2-b1)/(a1-a2)
    y = a1*x + b1
    return x,y
def MinProjectionOrthogonal(lb, lcoef, xb, yb):
    minDist = 1000000
    minLink = "-1"
    k = len(lb)
    for i in range(k):
        a = lcoef[i]
        b = lcoef[i+1]
        ap = -1/a
        bp = yb - ap*xb
        x,y = IntersectionDroite(a,b,ap,bp)
        dist = math.sqrt((xb-x)*(xb-x)+(yb-y)*(yb-y))
        if(minDist>dist):
            minDist=dist
            minLink=lb[i]
    return minLink

# Main function

In [ ]:
def AffectationBornes(dflink,dfborne,rayon):
    n = len(dfborne)
    m = len(dflink)
    dfborne["link"] = "" # link to affect station
    
    for i in range(n): # Go to each charging point to get its coordinates
        xb = float(dfborne.loc[i,"X"])
        yb = float(dfborne.loc[i,"Y"])
        lb=[]
        lcoef=[]
        for j in range(m): # Go to each links
            x1,y1,x2,y2 = GetCoordonnees(dflink,j) # 1/ Get coordinates of 2 nodes of each link
            a,b = Equationdroite(x1,x2,y1,y2)      # 2/ Calculate the coefficients of the line of the link
            
            s = isIntersection(xb, yb, rayon, a, b)# 3/ Test if line of the link touch circle C(xb,yb,rayon) 
            if(s):
                v = testCos(xb,yb,x1,y1,x2,y2)     # 4/ For testing if the angle at the terminal with the vertices of the potential link is obtuse (>90)
                ang = (v*180)/math.pi
                if(ang>=90):# angle obtu
                    lcoef.append(a)
                    lcoef.append(b)
                    lb.append(dflink.loc[j, "link_id"])             # list of potential links
        idMin = MinProjectionOrthogonal(lb, lcoef, xb, yb) # 5/ Check the link more near to the charging station
        dfborne.loc[i,"link"] = idMin
        print(i,n,dfborne.loc[i,"link"],dfborne.loc[i,"code_insee"],dfborne.loc[i,"id_station"])
    return dfborne

#BornesAffect = AffectationBornes(carLinkXY.copy(),Bdk.copy(),100) # BdK if reduce area to dunkerque
BornesAffect = AffectationBornes(carLinkXY.copy(),Bnpdc.copy(),100)# choose r to large to have links in the area
BornesAffect.to_csv("BornesAffected.csv", sep=";", encoding="utf8")

In [22]:
BA=pd.read_csv("BornesAffected.csv", ";",encoding='utf8', dtype=str) 

In [42]:
BA1= BA[["id_station","link","puiss_max","nbre_pdc","id_pdc","code_insee_","X","Y"]]

In [43]:
BA2= BA1[BA1["link"]!="-1"]

In [44]:
BA2['puiss_max'] = BA2['puiss_max'].astype(float, errors = 'raise')

/tmp/ipykernel_30786/3807389875.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BA2['puiss_max'] = BA2['puiss_max'].astype(float, errors = 'raise')


In [58]:
BA3 = BA2.copy().fillna(22.00)

In [61]:
BA3.to_csv("BornesAffectedReady4Awk.csv", sep=";", encoding="utf8") # see script chargersMaking.awk

# Bornes non affectées

In [236]:
NotAffected = BornesAffect[BornesAffect["link"]=="-1"] # 140 stations
NotAffected.reset_index(drop=True, inplace=True)
NotAffected.to_csv("BornesNotAffected.csv", sep=";", encoding="utf8")

In [240]:
def AffectationRestBornes(dflink,dfborne,rayon):
    n = len(dfborne)
    m = len(dflink)
    dfborne["link"] = "" # link to affect station
    
    for i in range(n): # Go to each charging point to get its coordinates
        xb = float(dfborne.loc[i,"X"])
        yb = float(dfborne.loc[i,"Y"])
        lb=[]
        lcoef=[]
        for j in range(m): # Go to each links
            x1,y1,x2,y2 = GetCoordonnees(dflink,j) # 1/ Get coordinates of 2 nodes of each link
            a,b = Equationdroite(x1,x2,y1,y2)      # 2/ Calculate the coefficients of the line of the link
            
            s = isIntersection(xb, yb, rayon, a, b)# 3/ Test if line of the link touch circle C(xb,yb,rayon) 
            if(s):
                v = testCos(xb,yb,x1,y1,x2,y2)     # 4/ For testing if the angle at the terminal with the vertices of the potential link is obtuse (>90)
                ang = (v*180)/math.pi
                if(ang<90):# angle obtu
                    lcoef.append(a)
                    lcoef.append(b)
                    lb.append(dflink.loc[j, "link_id"])             # list of potential links
        idMin = MinProjectionOrthogonal(lb, lcoef, xb, yb) # 5/ Check the link more near to the charging station
        dfborne.loc[i,"link"] = idMin
        print(i,n,dfborne.loc[i,"link"],dfborne.loc[i,"code_insee"],dfborne.loc[i,"id_station"])
    return dfborne

#BornesAffect = AffectationBornes(carLinkXY.copy(),Bdk.copy(),100) # BdK if reduce area to dunkerque
BornesAffect2 = AffectationRestBornes(carLinkXY.copy(),NotAffected.copy(),100)# choose r to large to have links in the area
#BornesAffect.to_csv("BornesAffected.csv", sep=";", encoding="utf8")

<ipython-input-223-4a5685968f1f>:11: RuntimeWarning: invalid value encountered in double_scalars
  cd = (y2-y1)/(x2-x1)


0 140 339103 59170 FR*G47*PLEMPEREURFIAT59187*1
1 140 339103 59170 FR*G47*PLEMPEREURFIAT59187*1


<ipython-input-223-4a5685968f1f>:34: RuntimeWarning: invalid value encountered in arccos
  return (np.arccos(alpha))


2 140 11896 59170 FR*G40*PLEMPEREURHYUNDAI59187*1
3 140 492024 59350 FR*SSD*PGHICLSAINTPHILIBERT*59160
4 140 492024 59350 FR*SSD*PGHICLSAINTPHILIBERT*59160
5 140 492024 59350 FR*SSD*PGHICLSAINTPHILIBERT*59160
6 140 492024 59350 FR*SSD*PGHICLSAINTPHILIBERT*59160
7 140 85056 59178 FR*E45*PIMTDOUAI59508*2
8 140 78978 59009 FR*SSD*PCEETRUS59656*4
9 140 466650 59088 FR*M59*P59088*001
10 140 33690 62193 FR*G44*PEUROPAUTOOPEL62100*1
11 140 271626 62250 FR*G44*PLEMPEREUROPEL62710*1
12 140 498492 59599 FR*E11*PLMTOURCOING59212*1


KeyboardInterrupt: 